In [1]:
import pandas as pd
import numpy as np

documents used:

emails.csv --> https://www.kaggle.com/datasets/jackksoncsie/spam-email-dataset
    Not the best dataset to train a naive bayes model, it contains emails with words slightly altered so as to not be detected easily

google-10000-english-usa.txt --> https://github.com/first20hours/google-10000-english
    this is a dataset of the top 10000 most frequently used english words(it has n = 9989 words). 




words 
--> a set that has all the words from the top 10k words dataset (n)

words_list
 --> used to iterate and have an orderly version of the words, this is used only for to make the features_vector  (1 x n)

emails
--> features (m x 1)

lables 
--> 1 if spam, 0 if ham (m x 1)



In [2]:
words = set(np.genfromtxt("google-10000-english-usa.txt", dtype=str, delimiter="\n"))

words_list = sorted(words)

emails = pd.read_csv('emails.csv')

lables = emails['spam']

emails = emails.drop('spam', axis=1)

words

{'rod',
 'halloween',
 'lawyer',
 'maintained',
 'kenya',
 'header',
 'detected',
 'sv',
 'toolkit',
 'deadline',
 'andale',
 'samples',
 'whilst',
 'prague',
 'locate',
 'ranging',
 'pending',
 'louise',
 'albany',
 'warriors',
 'guarantees',
 'surplus',
 'midnight',
 'nav',
 'attachments',
 'signals',
 'slowly',
 'occurrence',
 'hear',
 'growth',
 'oman',
 'dx',
 'levels',
 'violin',
 'fires',
 'genres',
 'circus',
 'dui',
 'agents',
 'all',
 'drivers',
 'annual',
 'bug',
 'lucia',
 'brad',
 'differently',
 'term',
 'aberdeen',
 'configurations',
 'escort',
 'simple',
 'integration',
 'victor',
 'viking',
 'worlds',
 'light',
 'many',
 'smithsonian',
 'urge',
 'kill',
 'separately',
 'curious',
 'exams',
 'keywords',
 'fa',
 'hide',
 'personally',
 'aaron',
 'isbn',
 'buses',
 'prev',
 'ends',
 'polyphonic',
 'generating',
 'examines',
 'advice',
 'launched',
 'creations',
 'affair',
 'seminar',
 'designer',
 'relates',
 'knowing',
 'mason',
 'posing',
 'bloggers',
 'parcel',
 'false

email_words 
--> a numpy array of python sets. each cell of the numpy array corrosponds to a different training example. each set in that cell contains all the unique words used in each email corrosponding to the cell. (contains m sets in m cells)

In [3]:
email_words = np.array([{'a'}], dtype=object)

for email in emails['text']:
        email_words = np.append(email_words, np.array([set(email.lower().split())], dtype=object), axis=0)


email_words = np.delete(email_words, 0, axis=0)

feature_vector is an (m x n) sparce matrix. each row corresponds to a different training example (different email). each column corrosponds to a different word in the word_list defined before. if the training example contains the word in its set, we mark the corrosponding column of the email's row as 1, if it does not contain the word in its set, we mark it as 0. 

we use a sparce matrix beacause majority of the values in the feature_vector is going to be 0s 

In [4]:
from scipy.sparse import csr_matrix, vstack

email_words = set([])
intersection_set = set([])

feature_vector = csr_matrix((0, len(words_list)))

for email in emails["text"]:
    email_words = set(email.lower().split())

    intersection_set = email_words & words
    
    row = []

    for word in words_list:
        if word in intersection_set:
            row.append(True) # appends 1
        else:
            row.append(False) # appends 0

    
    row = csr_matrix([row])
    feature_vector = vstack([feature_vector, row])

feature_vector = csr_matrix(feature_vector)

feature_vector.toarray()



    

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

we need to seperate the ham and spam emails from the features (emails) datset. we do this beacuse naive bayes is a generative learning algorithm and not a discriminative learning algorithm. 

In [5]:
ham_mail = feature_vector[lables.to_numpy() == 0]
spam_mail = feature_vector[lables.to_numpy() == 1]


In [6]:
spam_mail.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [7]:
ham_mail.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

We calculate the paramteres next: 

phi_y1 -->  P(Y = 1) probability of the email being a spam mail when selected at random from the dataset
phi_y0 -->  P(Y = 0) probability of the email being a ham mail when selected at random from the dataset

parameters_11 --> P(X~j~ = 1 | Y = 1) (1 x n)
parameters_01 --> P(X~j~ = 0 | Y = 1) (1 x n)
parameters_10 --> P(X~j~ = 1 | Y = 0) (1 x n)
parameters_00 --> P(X~j~ = 0 | Y = 0) (1 x n)

basically what the above 4 parameteres calculate is: 
what fraction of spam/ham emails had the word X~j~ in it? 


We will be using log probabilities instead of normal probabilities as it prevents underflow and also beacuse multiplication takes a lot of comutational power as compared to addition. 

since we will be using the log values of the parameters and the priors, we cannot let the values be 0, this would throw an error, we assign the 0 values to a very very tiny value (1e-10)

In [8]:
phi_y1 = spam_mail.shape[0] / (spam_mail.shape[0] + ham_mail.shape[0])

parameters_11 = spam_mail.sum(axis=0) / spam_mail.shape[0]

phi_y0 = 1 - phi_y1

parameters_01 = 1 - parameters_11

parameters_00 = ham_mail.sum(axis=0) / ham_mail.shape[0]

parameters_10 = 1 - parameters_00

# we cannot have 0 as a value in paramters, so i will make it a very tiny value isntead. 

parameters_11[parameters_11 == 0] = 1e-10
parameters_00[parameters_00 == 0] = 1e-10
parameters_01[parameters_01 == 0] = 1e-10
parameters_10[parameters_10 == 0] = 1e-10


In [9]:
parameters_00, parameters_01, parameters_10, parameters_11

(matrix([[7.82798165e-01, 6.88073394e-03, 1.37614679e-03, ...,
          6.88073394e-04, 6.88073394e-04, 1.00000000e-10]]),
 matrix([[0.27046784, 0.99853801, 1.        , ..., 1.        , 0.99926901,
          1.        ]]),
 matrix([[0.21720183, 0.99311927, 0.99862385, ..., 0.99931193, 0.99931193,
          1.        ]]),
 matrix([[7.29532164e-01, 1.46198830e-03, 1.00000000e-10, ...,
          1.00000000e-10, 7.30994152e-04, 1.00000000e-10]]))

In [10]:
parameters_11 = np.log(parameters_11)

parameters_01 = np.log(parameters_01)   

parameters_00 = np.log(parameters_00)   

parameters_10 = np.log(parameters_10)   



In [11]:
parameters_00, parameters_01, parameters_10, parameters_11

(matrix([[ -0.24488039,  -4.97902995,  -6.58846787, ...,  -7.28161505,
           -7.28161505, -23.02585093]]),
 matrix([[-1.30760209e+00, -1.46305805e-03,  0.00000000e+00, ...,
           0.00000000e+00, -7.31261459e-04,  0.00000000e+00]]),
 matrix([[-1.52692824e+00, -6.90451535e-03, -1.37709455e-03, ...,
          -6.88310226e-04, -6.88310226e-04,  0.00000000e+00]]),
 matrix([[ -0.31535182,  -6.52795792, -23.02585093, ..., -23.02585093,
           -7.2211051 , -23.02585093]]))

now that paramters are calculated, we can take in our input: query_email. 

we again make a set of the unique words in the query_email and store it in query_email_words

and we make a query_vector (1 x n) the same way we made the rows of the feature vector.

In [12]:

query_email = """ 
Subject: making room for " summer interns "  hello all :  i have some " good " news and i have some " bad " news . the " good " news  is that we are growing ! the " bad " news is that we are running out of space .  therefore , just for the summer we need to use jose ' s and joe ' s offices  on the 19 th floor for " summer " interns .  i will have to remove the extensions from those offices so they won ' t be  ringing all the time . however , we can put them back after the summer is  over .  i hope this is not too inconvenient for you , but with roman and jason needing  spaces on the 19 th floor , and another new hire coming on board , we are left  without any extra spaces .  thanks !  shirley

"""

query_email_words = set(query_email.lower().split())

query_vector = []
for word in words_list:
    if word in query_email_words:
        query_vector.append(1)
    else:
        query_vector.append(0)

query_vector = csr_matrix([query_vector])


In [13]:
query_vector.toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

calculating to give predictions:

log_ham_prob --> log probability of the mail being ham
log_spam_prob --> log probability of the mail being spam

In [14]:
log_ham_prob = np.dot(query_vector.toarray(), parameters_10.T) + np.dot((1 - query_vector.toarray()), parameters_00.T) + np.log(phi_y0)


log_spam_prob = np.dot(query_vector.toarray(), parameters_11.T) + np.dot((1 - query_vector.toarray()), parameters_01.T) + np.log(phi_y1)

normalizing the log_ham_prob beacuse the number of spammy words is significantly lesser than the ham words. hence we tend to always have high log_ham_prob

i came up with the number 500 after observing a few examples, there may be a better number for this. 

In [15]:
log_ham_prob /= 500

log_ham_prob

matrix([[-199.84669622]])

In [16]:
log_spam_prob

matrix([[-291.36752487]])

prediction: 

In [17]:
if log_spam_prob > log_ham_prob:
    print("spam")
else:
    print("ham")

ham
